In [1]:
from textgen.data.modules import SentenceCompletionIterableSplitFromStrings
from textgen.model.transformer import Transformer, TransformerModel
from textgen.model.pickers import ProbabilisticIndexPicker
from textgen.generation import TransformerSentenceIndicesGenerator
from pytorch_lightning import Trainer
import torch
import os

In [2]:
train_text = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis in sapien a turpis ullamcorper iaculis. 
                Vivamus ut mi sed nisl maximus vehicula. Aliquam non augue eget enim tempus posuere ut id dui. 
                Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. 
                Mauris sed bibendum lorem. Nam laoreet nibh ac rutrum ultrices. 
                Fusce quis porttitor orci, a vestibulum justo. Maecenas varius sapien justo, id fringilla risus lobortis rhoncus. 
                Vestibulum nec consectetur arcu. Maecenas varius non diam eu laoreet.
                Quisque eu mattis magna. Donec rhoncus ante sit amet augue egestas imperdiet. 
                Nulla facilisi. Vestibulum eget orci sed ante euismod tempus sit amet eu ante. 
                Cras vestibulum eget eros non rhoncus. Curabitur sit amet nisl leo. 
                Mauris egestas sapien tristique sagittis ultricies. Cras cursus quam id dui cursus, nec tincidunt diam fermentum. 
                Cras a lectus lorem. Etiam vitae tincidunt odio. Aliquam sapien ex, eleifend at ante non, gravida vestibulum eros.
                Morbi et elementum lacus. Morbi vel sagittis urna. Sed eu scelerisque libero. 
                Suspendisse volutpat iaculis velit vel finibus. Pellentesque eleifend lacus ac auctor ultricies. 
                Vivamus fermentum pulvinar viverra. Pellentesque luctus facilisis lacinia. 
                Sed efficitur, tellus nec rutrum cursus, neque magna sollicitudin enim, quis efficitur massa justo et urna."""
val_text =   """Etiam eget ipsum tincidunt, lobortis metus id, eleifend arcu. Pellentesque consectetur placerat quam ut varius. 
                Proin porta, elit et volutpat accumsan, quam est eleifend nisi, in tincidunt lacus ipsum quis mauris. 
                Nulla condimentum eu nibh sed ornare."""
test_text =  """Etiam ligula velit, molestie et semper vel, finibus id sem. 
                Donec pharetra nisl erat, nec scelerisque neque congue non. Praesent blandit elit sed ipsum porttitor maximus. 
                Mauris quis ipsum mollis, ullamcorper ante sit amet, efficitur risus. 
                Maecenas ipsum arcu, aliquam ut orci non, laoreet porta risus. 
                Nullam euismod blandit libero, quis consectetur est lacinia sed. Quisque consectetur ante ut orci iaculis feugiat. 
                Mauris blandit, dui vel maximus luctus, ligula eros tempus mi, vitae dapibus dui elit in purus. 
                Maecenas sed euismod orci. Nunc auctor sit amet diam et malesuada. 
                Vestibulum eget lorem dapibus eros pellentesque rutrum. Aliquam gravida tellus vel porttitor tincidunt. 
                Vivamus libero lectus, egestas vitae gravida a, lobortis feugiat leo."""

In [3]:
max_length = 20
batch_size = 64
num_workers = os.cpu_count()

In [4]:
dm = SentenceCompletionIterableSplitFromStrings(train_text, val_text, test_text, 
                                                max_length=max_length, batch_size=batch_size, num_workers=num_workers)
vocab_size = len(dm.config.corpus.vocabulary)

In [5]:
d_model = 64
d_ff = 16
num_heads = 4
num_layers = 4
drop_out_rate = 0.01

In [6]:
t = Transformer(TransformerModel(vocab_size, vocab_size, d_model, d_ff, num_heads, num_layers, drop_out_rate, max_length),
                dm.config, ProbabilisticIndexPicker(), lr=0.01)

In [7]:
trainer = Trainer(max_epochs=10, gpus=int(torch.cuda.is_available()))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [8]:
trainer.fit(t, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | model  | TransformerModel | 246 K 
1 | loss   | NLLLoss          | 0     
2 | metric | Accuracy         | 0     
--------------------------------------------
246 K     Trainable params
0         Non-trainable params
246 K     Total params
0.988     Total estimated model params size (MB)


Epoch 0: : 5it [00:00,  8.64it/s, loss=4.22, v_num=2]                 
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Validating: 1it [00:00,  3.06it/s]
Epoch 0: : 7it [00:01,  6.04it/s, loss=4.22, v_num=2]
Validating: 3it [00:00,  3.28it/s]
Epoch 0: : 9it [00:01,  5.13it/s, loss=4.22, v_num=2]
Epoch 1: : 4it [00:00,  7.14it/s, loss=3.59, v_num=2]
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 1: : 6it [00:01,  4.70it/s, loss=3.59, v_num=2]
Validating: 2it [00:00,  2.26it/s]
Epoch 1: : 8it [00:01,  4.54it/s, loss=3.59, v_num=2]
Epoch 1: : 9it [00:02,  3.87it/s, loss=3.59, v_num=2]
Epoch 2: : 4it [00:00,  7.17it/s, loss=3.18, v_num=2]
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 2: : 6it [00:00,  6.96it/s, loss=3.18, v_num=2]
Validating: 2it [00:00,  3.39it/s]
Epoch 2: : 8it [00:01,  5.40it/s, loss=3.18, v_num=2]
Epoch 2: : 9it [00:01,  5.02it/s, loss=3.18, v_num=2]
Epoch 3: : 4it [00:00,  7.19it/s, loss=2.87, v_num=2]
Validatin

In [9]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 4it [00:03,  1.21it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 4.457944869995117, 'test_metric': 0.15907754004001617}
--------------------------------------------------------------------------------


[{'test_metric': 0.15907754004001617, 'test_loss': 4.457944869995117}]

In [10]:
t.eval()
generator = TransformerSentenceIndicesGenerator(t)

In [11]:
generator.generate("Lorem ipsum")

'Lorem Ipsum Ullamcorper Dui Cursus.'